In [2]:
import pandas as pd
import numpy as np
import requests
import streamlit as st
import streamlit.components.v1 as components
import folium
from streamlit_folium import st_folium
import branca
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
rdw = pd.read_csv('fuel_count.csv')
rdw.rename({'datum_eerste_toelating':'date_of_acceptance','benzine_count': 'gasoline_count', 'elektriciteit_count':'electric_count', 'waterstof_count':'hydrogen_count'}, inplace=True, axis=1)
rdw.columns = map(str.lower, rdw.columns)

In [4]:
#laadpaaldata
lpd = pd.read_csv('laadpaaldata.csv')
lpd.columns = map(str.lower, lpd.columns)
lpd = lpd[lpd['chargetime'] >= 0]

In [5]:
#opencharge dataset
response = requests.get('https://api.openchargemap.io/v3/poi/?output=json&countrycode=NL&maxresults=10000&compact=true&verbose=false&key=772ba88b-50db-49f1-b9f1-933c6053edd3')

json_lp = response.json()

opencharge = pd.json_normalize(json_lp)

df4 = pd.json_normalize(opencharge.Connections)
df5 = pd.json_normalize(df4[0])

ocm = pd.concat([opencharge, df5], axis=1)

ocm.columns = ocm.columns.str.replace('.', '_') 
ocm.columns = ocm.columns.str.replace(' ', '_')

ocm = ocm.loc[:,~ocm.columns.duplicated()].copy()
ocm = ocm[ocm['Quantity'].notna()]
ocm = ocm[ocm['NumberOfPoints'] != 0]
ocm = ocm[ocm['AddressInfo_Latitude'] > 50]
ocm = ocm.drop(columns=['Connections','AddressInfo_DistanceUnit', 'GeneralComments', 'AddressInfo_RelatedURL', 'UsageCost', 'AddressInfo_AddressLine2', 'AddressInfo_AccessComments', 
                        'AddressInfo_ContactTelephone1', 'AddressInfo_ContactEmail', 'AddressInfo_ContactTelephone2', 'OperatorsReference', 'DataProvidersReference',
                        'MetadataValues', 'DateLastConfirmed', 'Reference', 'Amps', 'Voltage', 'Comments'])

ocm.columns = map(str.lower, ocm.columns)

ocm['addressinfo_postcode'] = ocm['addressinfo_postcode'].str.replace('[a-zA-Z]', '')
ocm['addressinfo_postcode'] = ocm['addressinfo_postcode'].str.replace(' ', '')

/var/folders/0j/dnd5nn45099d8yt1603ysnm00000gn/T/ipykernel_3831/1996120351.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ocm.columns = ocm.columns.str.replace('.', '_')
/var/folders/0j/dnd5nn45099d8yt1603ysnm00000gn/T/ipykernel_3831/1996120351.py:26: FutureWarning: The default value of regex will change from True to False in a future version.
  ocm['addressinfo_postcode'] = ocm['addressinfo_postcode'].str.replace('[a-zA-Z]', '')


In [6]:
rdw['date_of_acceptance'] = rdw['date_of_acceptance'].astype('datetime64[ns]')

rdw['electric'] = rdw['electric_count']

rdw['non_electric'] = rdw['gasoline_count'] + rdw['diesel_count'] + rdw['hydrogen_count'] + rdw['alcohol_count'] + rdw['lpg_count'] + rdw['cng_count'] + rdw['lng_count']
    
rdw['electric_percentage'] = (rdw['electric'] / (rdw['electric'] + rdw['non_electric']) * 100)

rdw = rdw[rdw['date_of_acceptance'] >= '2010-01-01']
rdw = rdw[rdw['date_of_acceptance'] <= '2022-12-31']

In [7]:
input1 = rdw['electric'].to_list()

In [8]:
X1 = rdw['date_of_acceptance']
y1 = np.log10(input1)

model = LinearRegression()
model.fit(X1.values.reshape(-1,1), y1)

X_predict1 = pd.date_range(start='2023-01-01', end='2051-01-01', freq='MS')
y_predict1 = model.predict(X_predict1.values.reshape(-1,1).astype("float64"))

In [9]:
new_electric = 10**(y_predict1)

In [10]:
df = pd.DataFrame()

In [11]:
df['pre_electric'] = new_electric.tolist()

In [12]:
input2 = rdw['non_electric'].to_list()

In [13]:
X2 = rdw['date_of_acceptance']
y2 = np.log10(input2)

model = LinearRegression()
model.fit(X2.values.reshape(-1,1), y2)

X_predict2 = pd.date_range(start='2023-01-01', end='2051-01-01', freq='MS')
y_predict2 = model.predict(X_predict2.values.reshape(-1,1).astype("float64"))

In [14]:
new_non_electric = 10**(y_predict2)

In [15]:
df['pre_non_electric'] = new_non_electric.tolist()

In [16]:
df['percentage'] = (df['pre_electric'] / (df['pre_electric'] + df['pre_non_electric']) * 100)

In [17]:
df['date'] = X_predict2

In [18]:
px.line(x = df.date, y = df.percentage)